In [1]:
import pandas as pd
from utils import heuristic_similarity

In [2]:
df = pd.read_csv('dataset/movies_all_data2.csv', encoding='utf-8')
df = df.dropna()
df.shape

(20760, 18)

In [3]:
df.sort_values(by='vote_count', ascending=False).head(20)

,adult,id,original_language,original_title,overview,popularity,runtime,tagline,title,vote_average,vote_count,genres,release_year,overview_keywords,tags,directors,characters,actors
9800,False,27205,en,Inception,"Cobb, a skilled thief who commits corporate es...",29.108149,148.0,Your mind is the scene of the crime.,Inception,8.1,14075.0,"['Action', 'Thriller', 'Science Fiction', 'Mys...",2010.0,"['cobb', 'skilled', 'thief', 'commit', 'corpor...","['loss of lover', 'dream', 'kidnapping', 'slee...",['Christopher Nolan'],"['Dom Cobb', 'Arthur', 'Ariadne']","['Leonardo DiCaprio', 'Joseph Gordon-Levitt', ..."
8335,False,155,en,The Dark Knight,Batman raises the stakes in his war on crime. ...,123.167259,152.0,Why So Serious?,The Dark Knight,8.3,12269.0,"['Drama', 'Action', 'Crime', 'Thriller']",2008.0,"['batman', 'raise', 'stake', 'war', 'crime', '...","['dc comics', 'crime fighter', 'secret identit...","['Christopher Nolan', 'Steve Gehrke', 'Nilo Ot...","['Bruce Wayne / Batman', 'Alfred Pennyworth', ...","['Christian Bale', 'Michael Caine', 'Heath Led..."
9364,False,19995,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",185.070892,162.0,Enter the World of Pandora.,Avatar,7.2,12114.0,"['Action', 'Adventure', 'Fantasy', 'Science Fi...",2009.0,"['nd', 'century', 'paraplegic', 'marine', 'dis...","['culture clash', 'future', 'space war', 'spac...",['James Cameron'],"['Jake Sully', 'Princess Neytiri', 'Dr. Grace ...","['Sam Worthington', 'Zoe Saldana', 'Sigourney ..."
10825,False,24428,en,The Avengers,When an unexpected enemy emerges and threatens...,89.887648,143.0,Some assembly required.,The Avengers,7.4,12000.0,"['Science Fiction', 'Action', 'Adventure']",2012.0,"['unexpected', 'enemy', 'emerge', 'threaten', ...","['new york', 'shield', 'marvel comic', 'superh...","['Joss Whedon', 'Trudy Ramirez']","['Tony Stark / Iron Man', 'Steve Rogers / Capt...","['Robert Downey Jr.', 'Chris Evans', 'Mark Ruf..."
14611,False,293660,en,Deadpool,Deadpool tells the origin story of former Spec...,187.860492,108.0,Witness the beginning of a happy ending,Deadpool,7.4,11444.0,"['Action', 'Adventure', 'Comedy']",2016.0,"['deadpool', 'tell', 'origin', 'story', 'speci...","['anti hero', 'mercenary', 'marvel comic', 'su...",['Tim Miller'],"['Wade Wilson / Deadpool', 'Vanessa Carlysle /...","['Ryan Reynolds', 'Morena Baccarin', 'Ed Skrein']"
13039,False,157336,en,Interstellar,Interstellar chronicles the adventures of a gr...,32.213481,169.0,Mankind was born on Earth. It was never meant ...,Interstellar,8.1,11187.0,"['Adventure', 'Drama', 'Science Fiction']",2014.0,"['interstellar', 'chronicle', 'adventure', 'gr...","['saving the world', 'artificial intelligence'...",['Christopher Nolan'],"['Joseph Cooper', 'Murph Cooper', 'Dr. Amelia ...","['Matthew McConaughey', 'Jessica Chastain', 'A..."
11778,False,68718,en,Django Unchained,"With the help of a German bounty hunter, a fre...",19.785025,165.0,"Life, liberty and the pursuit of vengeance.",Django Unchained,7.8,10297.0,"['Drama', 'Western']",2012.0,"['help', 'german', 'bounty', 'hunter', 'freed'...","['bounty hunter', 'hero', 'plantation', 'socie...","['Quentin Tarantino', 'Martin Kitrosser', 'Wil...","['Django', 'Dr. King Schultz', 'Calvin Candie']","['Jamie Foxx', 'Christoph Waltz', 'Leonardo Di..."
13444,False,118340,en,Guardians of the Galaxy,"Light years from Earth, 26 years after being a...",53.291601,121.0,All heroes start somewhere.,Guardians of the Galaxy,7.9,10014.0,"['Action', 'Science Fiction', 'Adventure']",2014.0,"['light', 'year', 'earth', 'year', 'abduct', '...","['marvel comic', 'spaceship', 'space', 'outer ...",['James Gunn'],"['Peter Quill / Star-Lord', 'Gamora', 'Drax th...","['Chris Pratt', 'Zoe Saldana', 'Dave Bautista']"
2192,False,550,en,Fight Club,A ticking-time-bomb insomniac and a slippery s...,63.869599,139.0,Mischief. Mayhem. Soap.,Fight Club,8.3,9678.0,['Drama'],1999.0,"['tickingtimebomb', 'insomniac', 'slippery', '...","['support group', 'dual identity', 'nihilism',..."

In [4]:
heuristic_similarity(df, 49051, 5)

search: The Hobbit: An Unexpected Journey


,id,title,genres,overview_keywords,tags,characters,directors,actors
0,122917,The Hobbit: The Battle of the Five Armies,"['Action', 'Adventure', 'Fantasy']","['immediately', 'event', 'desolation', 'smaug'...","['corruption', 'elves', 'dwarves', 'orcs', 'mi...","['Bilbo Baggins', 'Gandalf', 'Thorin Oakenshie...",['Peter Jackson'],"['Martin Freeman', 'Ian McKellen', 'Richard Ar..."
1,57158,The Hobbit: The Desolation of Smaug,"['Adventure', 'Fantasy']","['dwarf', 'bilbo', 'gandalf', 'successfully', ...","['elves', 'dwarves', 'orcs', 'hobbit', 'dragon...","['Bilbo Baggins', 'Gandalf', 'Thorin Oakenshie...","['Peter Jackson', 'Carolynne Cunningham', 'Ver...","['Martin Freeman', 'Ian McKellen', 'Richard Ar..."
2,120,The Lord of the Rings: The Fellowship of the Ring,"['Adventure', 'Fantasy', 'Action']","['young', 'hobbit', 'frodo', 'baggin', 'inheri...","['elves', 'dwarves', 'orcs', 'middle-earth (to...","['Frodo Baggins', 'Gandalf the Grey', 'Galadri...",['Peter Jackson'],"['Elijah Wood', 'Ian McKellen', 'Cate Blanchett']"
3,121,The Lord of the Rings: The Two Towers,"['Adventure', 'Fantasy', 'Action']","['frodo', 'sam', 'trek', 'mordor', 'destroy', ...","['elves', 'orcs', 'middle-earth (tolkien)', 'h...","['Frodo Baggins', 'Gandalf the White', 'Aragorn']",['Peter Jackson'],"['Elijah Wood', 'Ian McKellen', 'Viggo Mortens..."
4,122,The Lord of the Rings: The Return of the King,"['Adventure', 'Fantasy', 'Action']","['aragorn', 'reveal', 'heir', 'ancient', 'king...","['elves', 'orcs', 'middle-earth (tolkien)', 'b...","['Frodo Baggins', 'Gandalf the White', 'Aragorn']",['Peter Jackson'],"['Elijah Wood', 'Ian McKellen', 'Viggo Mortens..."


In [5]:
heuristic_similarity(df, "godfather", 5)

Found matching titles:
          id                    title
614      238            The Godfather
871      240   The Godfather: Part II
1457     242  The Godfather: Part III
5715   13398         Tokyo Godfathers
6099   29244             3 Godfathers
12753  77403          Disco Godfather


In [6]:
heuristic_similarity(df, "The Godfather", 5)

search: The Godfather


,id,title,genres,overview_keywords,tags,characters,directors,actors
0,10734,Escape from Alcatraz,"['Crime', 'Drama']","['escape', 'alcatraz', 'tell', 'story', 'man',...","['prison', 'based on novel', 'island', 'mouse'...","['Frank Morris', 'Warden', 'Chester ""Doc"" Dalt...","['Don Siegel', 'Lloyd Nelson', 'Luigi Alfano']","['Clint Eastwood', 'Patrick McGoohan', 'Robert..."
1,242,The Godfather: Part III,"['Crime', 'Drama', 'Thriller']","['midst', 'try', 'legitimize', 'business', 'de...","['italy', 'christianity', 'new york', 'assassi...","['Don Michael Corleone', 'Kay Adams Michelson'...",['Francis Ford Coppola'],"['Al Pacino', 'Diane Keaton', 'Andy García']"
2,240,The Godfather: Part II,"['Drama', 'Crime']","['continue', 'saga', 'corleone', 'crime', 'fam...","['italo-american', 'cuba', 'vororte', 'melanch...","['Don Michael Corleone', 'Tom Hagen', 'Kay Cor...","['Francis Ford Coppola', 'John Franco', 'Newt ...","['Al Pacino', 'Robert Duvall', 'Diane Keaton']"
3,10337,Bugsy,"['Drama', 'Crime']","['york', 'gangster', 'ben', 'bugsy', 'siegel',...","['underworld', 'hitman', 'biography', 'psychop...","[""Ben 'Bugsy' Siegel"", 'Virginia Hill', 'Micke...","['Barry Levinson', 'Julie Pitkanen']","['Warren Beatty', 'Annette Bening', 'Harvey Ke..."
4,190160,While the City Sleeps,"['Drama', 'Crime']","['tough', 'york', 'cop', 'determined', 'bring'...","['detective', 'melodrama', 'gangster']","['Dan Coghlan', 'Myrtle Sullivan', 'Marty']",['Jack Conway'],"['Lon Chaney', 'Anita Page', 'Carroll Nye']"


In [7]:
heuristic_similarity(df, 68721, 10)

search: Iron Man 3


,id,title,genres,overview_keywords,tags,characters,directors,actors
0,10138,Iron Man 2,"['Adventure', 'Action', 'Science Fiction']","['world', 'aware', 'dual', 'armored', 'superhe...","['malibu', 'marvel comic', 'superhero', 'based...","['Tony Stark / Iron Man', 'Virginia ""Pepper"" P...",['Jon Favreau'],"['Robert Downey Jr.', 'Gwyneth Paltrow', 'Don ..."
1,1726,Iron Man,"['Action', 'Science Fiction', 'Adventure']","['hold', 'captive', 'afghan', 'cave', 'billion...","['middle east', 'arms dealer', 'malibu', 'marv...","['Tony Stark / Iron Man', 'Lt. Col. James ""Rho...","['Jon Favreau', 'Cristina Weigmann']","['Robert Downey Jr.', 'Terrence Howard', 'Jeff..."
2,24428,The Avengers,"['Science Fiction', 'Action', 'Adventure']","['unexpected', 'enemy', 'emerge', 'threaten', ...","['new york', 'shield', 'marvel comic', 'superh...","['Tony Stark / Iron Man', 'Steve Rogers / Capt...","['Joss Whedon', 'Trudy Ramirez']","['Robert Downey Jr.', 'Chris Evans', 'Mark Ruf..."
3,1771,Captain America: The First Avenger,"['Action', 'Adventure', 'Science Fiction']","['predominantly', 'set', 'world', 'war', 'ii',...","['new york', 'usa', 'world war ii', 'nazis', '...","['Steve Rogers / Captain America', 'Johann Sch...",['Joe Johnston'],"['Chris Evans', 'Hugo Weaving', 'Tommy Lee Jon..."
4,271110,Captain America: Civil War,"['Adventure', 'Action', 'Science Fiction']","['follow', 'event', 'age', 'ultron', 'collecti...","['civil war', 'war', 'marvel comic', 'sequel',...","['Steve Rogers / Captain America', 'Tony Stark...","['Anthony Russo', 'Joe Russo', 'Joe Russo']","['Chris Evans', 'Robert Downey Jr.', 'Scarlett..."
5,16314,3 Ninjas,"['Action', 'Adventure', 'Comedy', 'Family']","['year', 'brother', 'samuel', 'jeffrey', 'mich...","['hero', 'rivalry', 'rescue', 'good vs evil', ...","['Grandpa Mori Tanaka', ""Samuel 'Rocky' Dougla...",['Jon Turteltaub'],"['Victor Wong', 'Michael Treanor', 'Max Elliot..."
6,305470,Power Rangers,"['Action', 'Adventure', 'Science Fiction']","['saban', 'power', 'ranger', 'follow', 'ordina...","['spaceship', 'based on tv series', 'tokusatsu...","['Jason Lee Scott / The Red Ranger', 'Kimberly...",['Dean Israelite'],"['Dacre Montgomery', 'Naomi Scott', 'RJ Cyler']"
7,18885,3 Ninjas Kick Back,"['Adventure', 'Action', 'Comedy', 'Family']","['championship', 'baseball', 'match', 'brother...","['underdog', 'hero', 'friendship', 'treasure h...","['Grandpa Mori Shintaro', ""Jeffrey 'Colt' Doug...",['Charles T. Kanganis'],"['Victor Wong', 'Max Elliott Slade', 'Sean Fox']"
8,141733,Cottage Country,"['Action', 'Comedy', 'Crime', 'Romance']","['todd', 'girlfriend', 'cammie', 'family', 'co...","['sex', 'canada', 'marriage proposal', 'canoe'...","['Cammie Ryan', 'Todd Chipowski', 'Salinger Ch...",['Peter Wellington'],"['Malin Åkerman', 'Tyler Labine', 'Dan Petroni..."
9,284052,Doctor Strange,"['Action', 'Adventure', 'Fantasy', 'Science Fi...","['career', 'destroy', 'brilliant', 'arrogant',...","['magic', 'marvel comic', 'superhero', 'based ...","['Dr. Stephen Strange', 'Mordo', 'Dr. Christin...","['Scott Derrickson', 'Jo Beckett', 'Michael Le...","['Benedict Cumberbatch', 'Chiwetel Ejiofor', '..."


In [8]:
heuristic_similarity(df, "The Lord of the Rings: The Return of the King", 15)

search: The Lord of the Rings: The Return of the King


,id,title,genres,overview_keywords,tags,characters,directors,actors
0,121,The Lord of the Rings: The Two Towers,"['Adventure', 'Fantasy', 'Action']","['frodo', 'sam', 'trek', 'mordor', 'destroy', ...","['elves', 'orcs', 'middle-earth (tolkien)', 'h...","['Frodo Baggins', 'Gandalf the White', 'Aragorn']",['Peter Jackson'],"['Elijah Wood', 'Ian McKellen', 'Viggo Mortens..."
1,120,The Lord of the Rings: The Fellowship of the Ring,"['Adventure', 'Fantasy', 'Action']","['young', 'hobbit', 'frodo', 'baggin', 'inheri...","['elves', 'dwarves', 'orcs', 'middle-earth (to...","['Frodo Baggins', 'Gandalf the Grey', 'Galadri...",['Peter Jackson'],"['Elijah Wood', 'Ian McKellen', 'Cate Blanchett']"
2,122917,The Hobbit: The Battle of the Five Armies,"['Action', 'Adventure', 'Fantasy']","['immediately', 'event', 'desolation', 'smaug'...","['corruption', 'elves', 'dwarves', 'orcs', 'mi...","['Bilbo Baggins', 'Gandalf', 'Thorin Oakenshie...",['Peter Jackson'],"['Martin Freeman', 'Ian McKellen', 'Richard Ar..."
3,5471,Beowulf & Grendel,"['Action', 'Adventure', 'Drama', 'Fantasy']","['bloodsoake', 'tale', 'norse', 'warrior', 'ba...","['hero', 'mythology', 'nordic mythology', 'myt...","['Beowulf', 'Grendels Father', 'Hrothgar']","['Sturla Gunnarsson', 'Linda Gibson']","['Gerard Butler', 'Spencer Wilding', 'Stellan ..."
4,17632,The Hunt for Gollum,"['Action', 'Adventure', 'Fantasy']","['british', 'fan', 'base', 'appendix', 'tolkie...","['middle-earth (tolkien)', 'the lord of the ri...","['Aragorn', 'Gandalf', 'Arithir']",['Chris Bouchard'],"['Adrian Webster', ""Patrick O'Connor"", 'Arin A..."
5,49051,The Hobbit: An Unexpected Journey,"['Adventure', 'Fantasy', 'Action']","['bilbo', 'baggin', 'hobbit', 'enjoy', 'quiet'...","['riddle', 'elves', 'dwarves', 'orcs', 'middle...","['Gandalf', 'Bilbo', 'Thorin']","['Peter Jackson', 'Victoria Sullivan']","['Ian McKellen', 'Martin Freeman', 'Richard Ar..."
6,36647,Blade,"['Horror', 'Action']","['blade', 'mother', 'bite', 'vampire', 'pregna...","['suicide', 'hero', 'vampire', 'bite', 'fistfi...","['Blade', 'Deacon Frost', 'Whistler']","['Stephen Norrington', 'Nicole Cummins', 'Mich...","['Wesley Snipes', 'Stephen Dorff', 'Kris Krist..."
7,10895,Pinocchio,"['Animation', 'Family']","['lonely', 'toymaker', 'geppetto', 'wish', 'an...","['italy', 'lie', 'magic', 'fairy', 'pinocchio'...","['Gideon', 'Carnival Barkers', 'John Worthingt...","['Hamilton Luske', 'Ben Sharpsteen']","['Mel Blanc', 'Don Brodie', 'Walter Catlett']"
8,27709,The Satan Bug,"['Mystery', 'Science Fiction', 'Thriller', 'Cr...","['satan', 'bug', 'science', 'fiction', 'govern...","['suicide', 'california', 'robbery', 'based on...","['Lee Barrett', 'Dr. Gregor Hoffman', 'Ann Wil...",['John Sturges'],"['George Maharis', 'Richard Basehart', 'Anne F..."
9,1911,The 13th Warrior,"['Adventure', 'Fantasy', 'Action']","['ad', 'arab', 'courtier', 'ahmad', 'ibn', 'fa...","['witch', 'cave', 'arabian', 'scandinavia', 'b...","['Ahmad Ibn Fadlān', 'Buliwyf', 'Herger the Jo...",['John McTiernan'],"['Antonio Banderas', 'Vladimir Kulich', 'Denni..."
